# 19.模型预测控制（model predictive control，MPC）算法
> 基于值函数的方法 DQN、基于策略的方法 REINFORCE 以及两者结合的方法 Actor-Critic，都是 **无模型（model-free）** 的方法，没有建立一个 **环境模型** 来帮助智能体决策。
> 在 **深度强化学习** 领域下，**基于模型（model-based）** 的方法不仅依赖于与 **真实环境** 交互获取的奖励信号，还试图用 **神经网络** 学习一个 **环境模型**，通过这个模型来**预测**环境的动态和奖励来帮助**智能体**训练和决策。
> 利用 **环境模型** 帮助智能体训练和决策的方法有很多种，例如类似之前 **Dyna** 的思想：生成一些数据来加入策略训练中。
> **模型预测控制** 算法并不构建一个显式的策略，而是使用 **环境模型** 预测未来一段时间内环境系统的状态变化，从而选择 **当前步** 要采取的动作。